In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [7]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001460                    1 -0.001059
0.000750        2 -0.001463                    2 -0.001050
0.001052        3 -0.001469                    3 -0.001067
0.001476        4 -0.001474                    4 -0.001070
0.002070        5 -0.001478                    5 -0.001070
0.002904        6 -0.001481                    6 -0.001069
0.004074        7 -0.001482                    7 -0.001067
0.005714        8 -0.001483                    8 -0.001064
0.008015        9 -0.001482                    9 -0.001059
0.011243       10 -0.001480                   10 -0.001055
0.015771       11 -0.001477                   11 -0.001056
0.022122       12 -0.001473                   12 -0.001053
0.031031       13 -0.001467                   13 -0.001047
0.043528       14 -0.001459                   14 -0.001036
0.061057       15 -0.001425                   15 -0.000998
0.085645       16 -0.001211                   16 -0.000825
0.120136       17 -0.000517                   17 -0.000291
0.168516       18  0.000921                   18  0.000790
0.236378       19  0.003498                   19  0.002686
0.331549       20  0.007776                   20  0.005778
0.465100       21  0.014508                   21  0.010671
0.652400       22  0.024650                   22  0.018645
0.915100       23  0.035665                   23  0.027467
1.283650       24  0.035566                   24  0.027956
1.800600       25  0.025161                   25  0.020669
2.525700       26  0.015606                   26  0.013497
3.542800       27  0.008918                   27  0.008068
4.969550       28  0.004607                   28  0.004292
6.970850       29  0.002009                   29  0.001793
9.778100       30  0.000454                   30  0.000150
13.715850      31 -0.000530                   31 -0.000962
19.239350      32 -0.001139                   32 -0.001672
26.987250      33 -0.001466                   33 -0.002082
37.855300      34 -0.001663                   34 -0.002341
53.100050      35 -0.001840                   35 -0.002532
73.887500      36 -0.002007                   36 -0.002672
97.662500      37 -0.002164                   37 -0.002794
121.437500     38 -0.002320                   38 -0.002885
145.212500     39 -0.002470                   39 -0.002894
168.987500     40 -0.002634                   40 -0.002981
192.762500     41 -0.002860                   41 -0.003144
216.537500     42 -0.003086                   42 -0.003493
240.312500     43 -0.003223                   43 -0.003738
264.087500     44 -0.003250                   44 -0.003840
287.862500     45 -0.003163                   45 -0.003802
311.637500     46 -0.002955                   46 -0.003637
335.412500     47 -0.002621                   47 -0.003285
359.187500     48 -0.002159                   48 -0.002819
382.962500     49 -0.001568                   49 -0.002225
406.737500     50 -0.000850                   50 -0.001478
430.512500     51 -0.000014                   51 -0.000593
454.287500     52  0.000931                   52  0.000407
478.062500     53  0.001974                   53  0.001530
501.837500     54  0.003111                   54  0.002773
525.612500     55  0.004344                   55  0.004094
549.387500     56  0.005678                   56  0.005532
573.162500     57  0.007117                   57  0.007097
596.937500     58  0.008662                   58  0.008795
620.712500     59  0.010315                   59  0.010629
644.487500     60  0.012078                   60  0.012550
668.262500     61  0.013950                   61  0.014561
692.037500     62  0.015933                   62  0.016696
715.812500     63  0.018030                   63  0.018966
739.587500     64  0.020237                   64  0.021361
763.362500     65  0.022550                   65  0.023870
78

# Fluxes by Layer

CRD                          CLIRAD             \
                        flug          fldg      fnetg      i       flug   
pressure    level                                                         
0.000000    1     -30.535477  0.000000e+00 -30.535477     11 -30.536835   
0.000624    2     -30.535478  5.437949e-08 -30.535478     11 -30.536835   
0.000876    3     -30.535478  7.668602e-08 -30.535478     11 -30.536835   
0.001229    4     -30.535478  1.092122e-07 -30.535478     11 -30.536835   
0.001723    5     -30.535478  1.567645e-07 -30.535478     11 -30.536835   
0.002417    6     -30.535478  2.262599e-07 -30.535478     11 -30.536835   
0.003391    7     -30.535478  3.278190e-07 -30.535478     11 -30.536835   
0.004757    8     -30.535479  4.762196e-07 -30.535478     11 -30.536836   
0.006672    9     -30.535479  6.930237e-07 -30.535479     11 -30.536836   
0.009359    10    -30.535480  1.009730e-06 -30.535479     11 -30.536837   
0.013128    11    -30.535481  1.472307e-06 -30.535480     11 -30.536838   
0.018415    12    -30.535483  2.147858e-06 -30.535481     11 -30.536839   
0.025830    13    -30.535485  3.134314e-06 -30.535482     11 -30.536840   
0.036232    14    -30.535488  4.574624e-06 -30.535484     11 -30.536843   
0.050823    15    -30.535493  6.682765e-06 -30.535486     11 -30.536846   
0.071291    16    -30.535500  1.002545e-05 -30.535490     11 -30.536851   
0.100000    17    -30.535510  1.651426e-05 -30.535494     11 -30.536859   
0.140271    18    -30.535527  3.070991e-05 -30.535496     11 -30.536871   
0.196760    19    -30.535552  6.156025e-05 -30.535490     11 -30.536888   
0.275997    20    -30.535585  1.274958e-04 -30.535457     11 -30.536911   
0.387100    21    -30.535621  2.658717e-04 -30.535355     11 -30.536934   
0.543100    22    -30.535638  5.508241e-04 -30.535087     11 -30.536943   
0.761700    23    -30.535571  1.122685e-03 -30.534449     11 -30.536895   
1.068500    24    -30.535309  2.156319e-03 -30.533153     11 -30.536718   
1.498800    25    -30.534879  3.539622e-03 -30.531340     11 -30.536455   
2.102400    26    -30.534475  4.934651e-03 -30.529541     11 -30.536261   
2.949000    27    -30.534190  6.214818e-03 -30.527976     11 -30.536222   
4.136600    28    -30.534076  7.354676e-03 -30.526721     11 -30.536394   
5.802500    29    -30.534179  8.366686e-03 -30.525812     11 -30.536833   
8.139200    30    -30.534551  9.295262e-03 -30.525256     11 -30.537605   
11.417000   31    -30.535270  1.019088e-02 -30.525080     11 -30.538807   
16.014700   32    -30.536453  1.108547e-02 -30.525368     11 -30.540569   
22.464000   33    -30.538252  1.201388e-02 -30.526238     11 -30.543062   
31.510500   34    -30.540889  1.307952e-02 -30.527809     11 -30.546562   
44.200100   35    -30.544734  1.442515e-02 -30.530309     11 -30.551535   
62.000000   36    -30.550325  1.613528e-02 -30.534190     11 -30.558613   
85.775000   37    -30.558053  1.821023e-02 -30.539842     11 -30.568207   
109.550000  38    -30.566190  2.025350e-02 -30.545936     11 -30.578100   
133.325000  39    -30.574861  2.239105e-02 -30.552470     11 -30.588319   
157.100000  40    -30.584025  2.459954e-02 -30.559425     11 -30.598573   
180.875000  41    -30.593832  2.698786e-02 -30.566845     11 -30.609219   
204.650000  42    -30.604934  3.003487e-02 -30.574899     11 -30.620851   
228.425000  43    -30.617959  3.436800e-02 -30.583591     11 -30.634658   
252.200000  44    -30.633022  4.035504e-02 -30.592667     11 -30.650701   
275.975000  45    -30.650088  4.826658e-02 -30.601821     11 -30.668827   
299.750000  46    -30.669092  5.836256e-02 -30.610730     11 -30.688893   
323.525000  47    -30.689948  7.089661e-02 -30.619052     11 -30.710839   
347.300000  48    -30.712548  8.611480e-02 -30.626434     11 -30.734249   
371.075000  49    -30.736768  1.042531e-01 -30.632515     11 -30.759110   
394.850000  50    -30.762466  1.255336e-01 -30.636932     11 -30.785421   
418.625000  51    -30.789475  1.501481e-01 -30.639

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')